In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
from matplotlib.colors import ListedColormap, Normalize
import numpy as np

## USA Statistics Per Party

In [18]:
congresses =  ['095', '096', '097','098', '099', '100', '101', '102', '103','104',
               '105', '106', '107','108', '109', '110', '111', '112','113', '114',
               '115', '116','117','118'] 

input_folder = "Data/USA/Filtered"
output_folder = "Images/heatmaps_party_agreement_USA"
os.makedirs(output_folder, exist_ok=True)

for congress in congresses:
    file_path = f"{input_folder}/H{congress}_filtered_USA_votes.csv"
    df = pd.read_csv(file_path)

    # Only keep 'yes' and 'no' votes from two main parties
    df = df[df["cast_code"].isin([1, 2])]
    df = df[df["party_code"].isin([100, 200])]

    vote_counts = df.groupby(["party_code", "rollnumber", "cast_code"]).size().unstack(fill_value=0)
    vote_counts.columns = ["no_votes", "yes_votes"]
    vote_counts["total_votes"] = vote_counts["no_votes"] + vote_counts["yes_votes"]
    vote_counts["agreement_rate"] = vote_counts[["no_votes", "yes_votes"]].max(axis=1) / vote_counts["total_votes"]

    pivot_data = vote_counts.reset_index().pivot_table(
        index="rollnumber", columns="party_code", values="agreement_rate", fill_value=0
    )

    # Create a masked array to handle zeros
    masked_data = pivot_data.copy()
    mask = masked_data == 0

    # Create a colormap where values == 0 appear black
    cmap = sns.color_palette("coolwarm", as_cmap=True)
    cmap = cmap(np.linspace(0, 1, 256))
    cmap[0] = [0, 0, 0, 1]  # Set the first (lowest) color to black
    custom_cmap = ListedColormap(cmap)

    # Normalize the data, ensuring 0 maps to index 0 in colormap
    from matplotlib.colors import Normalize
    norm = Normalize(vmin=0, vmax=1)

    # Plot and save
    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot_data, cmap=custom_cmap, norm=norm, cbar=True)
    plt.title(f"Party Agreement Rate by Roll Call – {congress}th Congress")
    plt.xlabel("Party Code")
    plt.ylabel("Roll Number")

    output_path = os.path.join(output_folder, f"agreement_heatmap_congress{congress}.png")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

## Danish Statistics Per Party

In [17]:
congresses = ['01_05','05_07','07_11','11_15','15_19','19_22']
input_folder = "Data/Denmark/Raw"
output_folder = "Images/heatmaps_party_agreement_danish"
os.makedirs(output_folder, exist_ok=True)

for congress in congresses:
    input_votes = f"{input_folder}/P{congress}_DK.csv"
    df = pd.read_csv(input_votes)

    # Filter only Yes (1) and No (2) votes
    df = df[df["typeid_x"].isin([1, 2])]

    # Group and count votes
    vote_counts = df.groupby(["party", "afstemningid", "typeid_x"]).size().unstack(fill_value=0)

    # Rename columns
    vote_counts.columns = ["no_votes", "yes_votes"]

    # Calculate totals and agreement rates
    vote_counts["total_votes"] = vote_counts["no_votes"] + vote_counts["yes_votes"]
    vote_counts["agreement_rate"] = vote_counts[["no_votes", "yes_votes"]].max(axis=1) / vote_counts["total_votes"]

    # Pivot data for heatmap
    pivot_data = vote_counts.reset_index().pivot_table(
        index="afstemningid", columns="party", values="agreement_rate", fill_value=0
    )

    # Create a custom colormap: 0 values -> black, rest -> coolwarm
    base_cmap = sns.color_palette("coolwarm", as_cmap=True)
    base_array = base_cmap(np.linspace(0, 1, 256))
    base_array[0] = [0, 0, 0, 1]  # Set first color (0) to black
    custom_cmap = ListedColormap(base_array)

    norm = Normalize(vmin=0, vmax=1)

    # Plot and save
    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot_data, cmap=custom_cmap, norm=norm, cbar=True)
    plt.title(f"Party Agreement Rate by Roll Call – Denmark {congress}")
    plt.xlabel("Party")
    plt.ylabel("Roll Number")

    output_path = os.path.join(output_folder, f"agreement_heatmap_dk_{congress}.png")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
